<a href="https://colab.research.google.com/github/Udit-dutta/Deep-Extreme-Image-DeiT-model-quantized-to-8-bit-integer-INT8-/blob/main/COCO_YOLO_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch
import os
import json
import time
import cv2
import shutil
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
try:
    from pycocotools.coco import COCO
except ModuleNotFoundError:
    import subprocess
    subprocess.check_call(["pip", "install", "pycocotools"])
    from pycocotools.coco import COCO

In [ ]:
!mkdir -p /mnt/data/coco_subset/
!cd /mnt/data/coco_subset/ && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip /mnt/data/coco_subset/annotations_trainval2017.zip -d /mnt/data/coco_subset/

--2025-03-16 17:03:58--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.172, 16.15.176.73, 16.15.178.100, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.172|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  96.3MB/s    in 2.5s    

2025-03-16 17:04:01 (96.3 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  /mnt/data/coco_subset/annotations_trainval2017.zip
  inflating: /mnt/data/coco_subset/annotations/instances_train2017.json  
  inflating: /mnt/data/coco_subset/annotations/instances_val2017.json  
  inflating: /mnt/data/coco_subset/annotations/captions_train2017.json  
  inflating: /mnt/data/coco_subset/annotations/captions_val2017.json  
  inflating: /mnt/data/coco_subset/annotations/p

In [ ]:
# Create dataset directory
!mkdir -p /mnt/data/coco_subset/

# Download COCO validation images
!wget -c http://images.cocodataset.org/zips/val2017.zip -O /mnt/data/coco_subset/val2017.zip

# Unzip images
!unzip -q /mnt/data/coco_subset/val2017.zip -d /mnt/data/coco_subset/


--2025-03-16 17:04:10--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.125.169, 52.217.201.41, 54.231.131.241, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.125.169|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘/mnt/data/coco_subset/val2017.zip’

/mnt/data/coco_subs 100%[===================>] 777.80M  95.4MB/s    in 8.1s    

2025-03-16 17:04:18 (95.6 MB/s) - ‘/mnt/data/coco_subset/val2017.zip’ saved [815585330/815585330]



In [ ]:
# Define dataset paths
unzipped_folder = "/mnt/data/coco_subset/"
anno_file = os.path.join(unzipped_folder, 'annotations', 'instances_val2017.json')
image_dir = os.path.join(unzipped_folder, 'val2017')
subset_dir = os.path.join(unzipped_folder, 'subset')
os.makedirs(subset_dir, exist_ok=True)

In [ ]:
# Load COCO annotations
coco = COCO(anno_file)

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!


In [ ]:
# Select 10 categories, 100 images each
selected_categories = coco.getCatIds()[:10]
selected_images = set()
for cat in selected_categories:
    img_ids = coco.getImgIds(catIds=[cat])[:100]
    selected_images.update(img_ids)

In [ ]:
print(f"Total selected images: {len(selected_images)}")

Total selected images: 766


In [ ]:
for img_id in selected_images:
    img_info = coco.loadImgs([img_id])[0]
    src_path = os.path.join(image_dir, img_info['file_name'])
    dst_path = os.path.join(subset_dir, img_info['file_name'])

    print(f"Checking: {src_path} -> {dst_path}")

    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
        print(f"✅ Copied: {src_path} -> {dst_path}")
    else:
        print(f"❌ Missing: {src_path}")

Checking: /mnt/data/coco_subset/val2017/000000532481.jpg -> /mnt/data/coco_subset/subset/000000532481.jpg
✅ Copied: /mnt/data/coco_subset/val2017/000000532481.jpg -> /mnt/data/coco_subset/subset/000000532481.jpg
Checking: /mnt/data/coco_subset/val2017/000000184321.jpg -> /mnt/data/coco_subset/subset/000000184321.jpg
✅ Copied: /mnt/data/coco_subset/val2017/000000184321.jpg -> /mnt/data/coco_subset/subset/000000184321.jpg
Checking: /mnt/data/coco_subset/val2017/000000458755.jpg -> /mnt/data/coco_subset/subset/000000458755.jpg
✅ Copied: /mnt/data/coco_subset/val2017/000000458755.jpg -> /mnt/data/coco_subset/subset/000000458755.jpg
Checking: /mnt/data/coco_subset/val2017/000000184324.jpg -> /mnt/data/coco_subset/subset/000000184324.jpg
✅ Copied: /mnt/data/coco_subset/val2017/000000184324.jpg -> /mnt/data/coco_subset/subset/000000184324.jpg
Checking: /mnt/data/coco_subset/val2017/000000385029.jpg -> /mnt/data/coco_subset/subset/000000385029.jpg
✅ Copied: /mnt/data/coco_subset/val2017/000000

In [ ]:
print(f"Subset directory exists: {os.path.exists(subset_dir)}")
print(f"Files in subset_dir: {os.listdir(subset_dir)}")

Subset directory exists: True
Files in subset_dir: ['000000220732.jpg', '000000311909.jpg', '000000042563.jpg', '000000534827.jpg', '000000098392.jpg', '000000367680.jpg', '000000011511.jpg', '000000292997.jpg', '000000446206.jpg', '000000567640.jpg', '000000279278.jpg', '000000311518.jpg', '000000076547.jpg', '000000469192.jpg', '000000198928.jpg', '000000007816.jpg', '000000388903.jpg', '000000272136.jpg', '000000046497.jpg', '000000308587.jpg', '000000489014.jpg', '000000192607.jpg', '000000213033.jpg', '000000147729.jpg', '000000084752.jpg', '000000244379.jpg', '000000143556.jpg', '000000397351.jpg', '000000395633.jpg', '000000571943.jpg', '000000561335.jpg', '000000229747.jpg', '000000300659.jpg', '000000010363.jpg', '000000352491.jpg', '000000005477.jpg', '000000490413.jpg', '000000383621.jpg', '000000284725.jpg', '000000383289.jpg', '000000119516.jpg', '000000442456.jpg', '000000090208.jpg', '000000361103.jpg', '000000253002.jpg', '000000213086.jpg', '000000256195.jpg', '0000005

In [ ]:
import torch
import torchvision
import time
from torchvision import transforms
from PIL import Image
from timm import create_model

In [ ]:
# Load pretrained DeiT model
deit_model_fp32 = create_model("deit_base_patch16_224", pretrained=True)
deit_model_fp32.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [ ]:
# Convert to int8 using torch quantization
deit_model_int8 = torch.quantization.quantize_dynamic(
    deit_model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)
deit_model_int8.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): DynamicQuantizedLinear(in_features=768, out_features=2304, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): DynamicQuantizedLinear(in_features=768, out_features

In [ ]:
# Load COCO dataset
coco_val_new = torchvision.datasets.CocoDetection(
    root="/mnt/data/coco_subset/val2017", # Changed root to point to your subset directory
    annFile="/mnt/data/coco_subset/annotations/instances_val2017.json", # Changed annFile to point to your subset directory
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


In [ ]:
# Function to measure FPS
def measure_fps(model, dataset, num_samples=100):
    start_time = time.time()
    with torch.no_grad():
        for i in range(num_samples):
            img, _ = dataset[i]
            img = img.unsqueeze(0)
            _ = model(img)
    total_time = time.time() - start_time
    return num_samples / total_time

In [ ]:
# Measure FPS
fps_fp32 = measure_fps(deit_model_fp32, coco_val_new)
fps_int8 = measure_fps(deit_model_int8, coco_val_new)

In [ ]:
print(f"FPS (Float32): {fps_fp32:.2f}")
print(f"FPS (Int8): {fps_int8:.2f}")

FPS (Float32): 1.37
FPS (Int8): 2.50


In [ ]:
# Placeholder function for AP calculation (COCO eval requires detection outputs)
def compute_ap(model, dataset):
    # Normally, COCO evaluation requires bounding box outputs.
    # DeiT is a classification model, so AP is not directly applicable.
    # If using a detection model, you would run inference and compare with ground truth.
    return 0  # Placeholder

In [ ]:
ap_fp32 = compute_ap(deit_model_fp32, coco_val_new)
ap_int8 = compute_ap(deit_model_int8, coco_val_new)

In [ ]:
print(f"AP Score (Float32): {ap_fp32:.2f}")
print(f"AP Score (Int8): {ap_int8:.2f}")

AP Score (Float32): 0.00
AP Score (Int8): 0.00
